<a href="https://colab.research.google.com/github/AronBensimhon/RL_game/blob/main/Frozen_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install gym==0.26.2
! pip install pygame numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827725 sha256=55bc4e02281a809150173bcce193f63cd4f9bb4428360f58e950eeba1ad42df0
  Stored in directory: /root/.cache/pip/wheels/1c/77/9e/9af5470201a0b0543937933ee99ba884cd237d2faefe8f4d37
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [ ]:
!pip install gym==0.26.2
!pip install numpy==1.23.5
import os
os.kill(os.getpid(), 9)  # מרענן את הקרנל (חובה לשינוי ייכנס לתוקף)

In [14]:
import numpy as np
import gym
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
from gym.envs.registration import register
import random
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)

In [15]:
# ====================================================
# Cell 2: פונקציה ליצירת לוח אקראי (עד 3 חורים, עד 2 פרסים)
# ====================================================
def generate_custom_map(size=4, num_holes=3, num_rewards=2):
    grid = [['F'] * size for _ in range(size)]
    grid[0][0] = 'S'
    grid[-1][-1] = 'G'

    # בחר עד `num_holes` קואורדינטות לחורים, לא במקומות S או G
    holes = random.sample(
        [(i, j) for i in range(size) for j in range(size)
         if (i, j) not in [(0, 0), (size - 1, size - 1)]],
        k=min(num_holes, size*size-2)
    )
    for i, j in holes:
        grid[i][j] = 'H'

    # בחר עד `num_rewards` קואורדינטות לפרסים, רק בתאים 'F'
    rewards = random.sample(
        [(i, j) for i in range(size) for j in range(size) if grid[i][j] == 'F'],
        k=min(num_rewards, sum(1 for row in grid for c in row if c == 'F'))
    )
    for i, j in rewards:
        grid[i][j] = 'R'

    return [''.join(row) for row in grid]

In [16]:
# ====================================================
# Cell 3: רישום סביבה מותאמת עם מזהה ייחודי + render_mode='ansi'
# ====================================================
def make_custom_env(index=0):
    # יוצר מפה חדשה
    custom_map = generate_custom_map(size=4, num_holes=3, num_rewards=2)

    env_id = f"CustomFrozenLake-v{index}"
    register(
        id=env_id,
        entry_point="gym.envs.toy_text.frozen_lake:FrozenLakeEnv",
        kwargs={"desc": custom_map, "is_slippery": False},
        max_episode_steps=100,
        reward_threshold=0.78,
    )
    # render_mode='ansi' מאפשר env.render() להחזיר מחרוזת טקסט
    env = gym.make(env_id, render_mode="ansi")
    return env, custom_map

In [19]:
# ====================================================
# Cell 4: Wrapper לתגמול מותאם
# ====================================================
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env, reward_positions):
        super().__init__(env)
        self.reward_positions = reward_positions
        self.size = int(np.sqrt(env.observation_space.n))

    def step(self, action):
        # כאן קולט את כל החמישה ערכים שמחזירה env.step
        state, reward, done, truncated, info = self.env.step(action)
        r = -1  # כל צעד עולה נקודה
        row, col = divmod(state, self.size)
        if (row, col) in self.reward_positions:
            r += 3
        if reward == 1:  # הגעה ליעד
            r += 10
        # החזירו info (ולא inf)
        return state, r, done, truncated, info

In [ ]:
# ====================================================
# Cell 5: לולאת אימון Q-Learning + הדפסת כל 50 אפיזודות
# ====================================================
# Q-table
Q = np.zeros((16, 4))

# היפר־פרמטרים
alpha = 0.8
gamma = 0.95
epsilon = 0.1
episodes = 1000

success_per_50 = []
reward_per_50 = []

success_counter = 0
reward_accumulator = 0

for episode in range(1, episodes + 1):
    # יצירת סביבת RL חדשה לפרק הנוכחי
    env, custom_map = make_custom_env(index=episode)
    reward_positions = [
        (i, j) for i in range(4) for j in range(4) if custom_map[i][j] == 'R'
    ]
    env = CustomRewardWrapper(env, reward_positions)

    # אימון פרק
    state = env.reset()[0]
    done = False
    total_reward = 0
    step_count = 0

    while not done:
        # ε-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])

        next_state, reward, done, _, _ = env.step(action)
        Q[state, action] += alpha * (
            reward + gamma * np.max(Q[next_state]) - Q[state, action]
        )
        total_reward += reward
        state = next_state
        step_count += 1

    # עדכון מטריקות
    reward_accumulator += total_reward
    # הצלחה = הגעה ליעד (עטיפת תגמול מוסיפה ≥10 רק בהצלחה)
    if reward >= 10 and done:
        success_counter += 1

    # כל 50 פרקים – הדפס סיכום, מפה ו-rollout
    if episode % 50 == 0:
        print("\n" + "=" * 60)
        print(f"🎯 Episode {episode} | Successes: {success_counter}/50 | "
              f"Avg Reward: {reward_accumulator/50:.2f}")
        print(f"📘 Last episode steps: {step_count}\n")

        success_per_50.append(success_counter)
        reward_per_50.append(reward_accumulator / 50)
        success_counter = 0
        reward_accumulator = 0

        # הדפסת מפת הדוגמה
        print("🗺️ Sample map (S=start, G=goal, H=hole, R=reward):")
        for row in custom_map:
            print(" ".join(row))
        print()

        # rollout
        print("👣 Sample rollout:")
        state = env.reset()[0]
        done = False
        step_log = []

        while not done:
            action = np.argmax(Q[state])
            state, reward, done, _, _ = env.step(action)
            step_log.append(env.render())

        print(f"\n🕹️ Rollout steps: {len(step_log)}")
        if not step_log:
            print("⚠️ No rollout: the agent fell or ended immediately.")
        else:
            for i, frame in enumerate(step_log, start=1):
                print(f"\nStep {i}:\n{frame}")


🎯 Episode 50 | Successes: 0/50 | Avg Reward: 196.22
📘 Last episode steps: 2

🗺️ Sample map (S=start, G=goal, H=hole, R=reward):
S F H H
F H F F
R F F F
F F R G

👣 Sample rollout:

🕹️ Rollout steps: 2

Step 1:
  (Right)
SFHH
FHFF
RFFF
FFRG


Step 2:
  (Right)
SFHH
FHFF
RFFF
FFRG


🎯 Episode 100 | Successes: 0/50 | Avg Reward: 807.96
📘 Last episode steps: 670

🗺️ Sample map (S=start, G=goal, H=hole, R=reward):
S R F H
F F H F
R F F F
F H F G

👣 Sample rollout:


In [ ]:
# ====================================================
# Cell 6 (אופציונלי): גרפים של הצלחות ותגמולים
# ====================================================
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(50, episodes+1, 50), success_per_50, marker='o')
plt.title("Successes per 50 Episodes")
plt.xlabel("Episode")
plt.ylabel("Count")
plt.ylim(0, 50)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(50, episodes+1, 50), reward_per_50, marker='s', color='orange')
plt.title("Avg Reward per 50 Episodes")
plt.xlabel("Episode")
plt.ylabel("Avg Reward")
plt.grid(True)

plt.tight_layout()
plt.show()